## Main code

### Libraries and Modules

In [1]:
import sys
import math
import glob
import os
import time
import csv

import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi
import cv2

import import_ipynb
import voronoi_constructor as vor_set
import vertices_assembler as assembler
import utils

importing Jupyter notebook from voronoi_constructor.ipynb
importing Jupyter notebook from node_linker.ipynb
importing Jupyter notebook from pps.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from vertices_assembler.ipynb


### Global Parameters

In [2]:
parameters = {"robots"      : 5,
              "radius"      : 0.1,
              "movement"    : 0.1,
              "tolerance"   : 0.05,
              "deadlock_mov": 0.1,
              "previous"    : 1
             }

### General Options

In [6]:
opt = { "palette"    : utils.colors(parameters["robots"]),
        "verbose"    : 0,
        "optimize"   : 0,
        "plot"       : 0,
        "zoom"       : 0,
        "save_image" : 0,
        "save_data"  : 0,
        "experiments": 1,
        "seed"       : 0,
        "breakpoint" : 100,
        "see_more_steps" : 100,
        "algorithm"  : "aga",
        "path_data"  : "/home/leduin/Escritorio/Tesis/Resultados/Datos/",
        "path_image" : "/home/leduin/Escritorio/Tesis/Imagenes/"
       }

### Main Loop

In [4]:
def main_module(key):
    with open(opt["path_data"]+opt["algorithm"]+key+".csv","a") as f:
        if opt["save_data"]:
            csv_writer = csv.writer(f)
        for experiment in range(opt["experiments"]):
            # INITIALIZATION
            start_time = time.time()
            inst_experiment = experiment + opt["seed"]
            pos = utils.pos_gen(parameters["robots"], inst_experiment)
            vor_i = Voronoi(pos["i"])
            vor_fin  = Voronoi(pos["f"])
            robots_path = [] # To save the robots path along execution
            finished = np.all(pos["i"] == pos["f"]) 
            # To save the traveled distance of each robot 
            covered_dist = [0 for _ in range(parameters["robots"])]
            counter = 0 # For steps
            # COLLISION FREE CONFIGURATION
            if utils.collision(pos["i"], vor_i.ridge_points,
                               parameters["radius"]):
                print('ERROR!: Change the initial configuration')
                print("Positions:", vor_i.ridge_points)
                break
            if utils.collision(pos["f"], vor_fin.ridge_points,
                               parameters["radius"]):
                print('ERROR!: Change the final configuration')
                print("Positions:", vor_fin.ridge_points)
                break
            # MAIN LOOP
            while not finished:
                #Final iteration
                if np.all(pos["i"] == pos["f"]):
                    finished = True
                if opt["verbose"]:
                    print("==================================================")
                    print('                    MAIN DATA')
                    print("==================================================")
                    print('\nRobots:\n', pos["i"],'\n')
                    print('Final positions:\n', pos["f"],'\n')
                    print("Colors:")
                    for color in opt["palette"]:
                        print(color)
                    print("\n")
                # Generate Voronoi Diagram for the current positions
                vor_i = Voronoi(pos["i"])
                # Generate the Voronoi Diagram and get the approximated values
                # of inf vertices
                vor_prox = vor_set.vor_prox(pos["i"], vor_i, opt["verbose"])
                # Assemble the vertices and get the ordered vertices
                vrtx_assembled = assembler.main(parameters["robots"], vor_i,
                                                vor_prox["edge"],
                                                opt["verbose"])
                robots_path.append(pos["i"]) # Save the previous position
                # Generate the BVC and the closer points
                bvc = vor_set.bvc_gen(pos, parameters["radius"],
                                      vor_i.ridge_points, vor_prox,
                                      vrtx_assembled, robots_path, opt)
                utils.save_image(opt, parameters["robots"], counter)
                # DEADLOCK TESTING
                bvc = vor_set.deadlock_test(parameters, opt, bvc, pos, counter,
                                            robots_path)
                if opt["verbose"]:
                    print('\nRounds: ',counter)
                    print('\n*-*-*-*-*-*-*-*ENDING ROUND*-*-*-*-*-*-*-*-*\n')
                counter += 1
                if counter == opt["breakpoint"]:
                    opt["verbose"] = 1
                    opt["plot"] = 1
                if counter > opt["breakpoint"] + opt["see_more_steps"]:
                    opt["verbose"] = 0
                    opt["plot"] = 0
                # UPDATING POSITIONS
                covered_dist, pos = utils.update_pos(bvc, pos, parameters,
                                                     covered_dist)
            effectiveness = np.sum(pos["dist"]) / np.sum(covered_dist)
            if opt["save_data"]:
                csv_writer.writerow([parameters[key], inst_experiment, counter,
                                     time.time() - start_time, effectiveness])
            print('Experiment ' + str(inst_experiment) + " done!")

### Running for the stablished number of experiments

In [7]:
main_module("robots")

Experiment 0 done!


### One Parameter Analysis

In [7]:
parameter = "robots"
print("Working on",parameter,"...")
previous_value = parameters[parameter]
if parameter == "deadlock_mov":
    parameters[parameter] = 1
elif parameter == "tolerance":
    parameters[parameter] = 0.1
elif parameter == "movement":
    parameters[parameter] = 20
for i in range(2):
    print(parameter,"equal to",parameters[parameter],"...")
    main_module(parameter)
    if parameter == "tolerance":
        parameters[parameter] = round(parameters[parameter]+0.1,2)
    elif parameter == "movement":
        parameters[parameter] += 10 
    else:
        parameters[parameter] += 1
parameters[parameter] = previous_value

Working on robots ...
robots equal to 5 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 6 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 

### All Parameter Analysis

In [8]:
for parameter in parameters:
    print("Working on",parameter,"...")
    previous_value = parameters[parameter]
    if parameter == "deadlock_mov":
        parameters[parameter] = 1
    for i in range(10):
        print(parameter,"equal to",parameters[parameter],"...")
        main_module(parameter)
        parameters[parameter] += 1
    parameters[parameter] = previous_value

Working on robots ...
robots equal to 5 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!


voronoi_constructor.ipynb:16: RuntimeWarning: invalid value encountered in double_scalars
  },


Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 6 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment 4 done!
Experiment 5 done!
Experiment 6 done!
Experiment 7 done!
Experiment 8 done!
Experiment 9 done!
Experiment 10 done!
Experiment 11 done!
Experiment 12 done!
Experiment 13 done!
Experiment 14 done!
Experiment 15 done!
Experiment 16 done!
Experiment 17 done!
Experiment 18 done!
Experiment 19 done!
Experiment 20 done!
Experiment 21 done!
Experiment 22 done!
Experiment 23 done!
Experiment 24 done!
Experiment 25 done!
Experiment 26 done!
Experiment 27 done!
Experiment 28 done!
Experiment 29 done!
robots equal to 7 ...
Experiment 0 done!
Experiment 1 done!
Experiment 2 done!
Experiment 3 done!
Experiment

KeyboardInterrupt: 

### Generate Video

In [11]:
frameSize = (432, 288)

out = cv2.VideoWriter('/home/leduin/Desktop/Hub/Images/Simple/animation_simple_'
                      +str(robots)+'.mp4',
                      cv2.VideoWriter_fourcc(*'DIVX'), 4, frameSize)

for filename in sorted(glob.glob('/home/leduin/Desktop/Hub/Images/Simple/'
                                 +str(robots)+'/*.png'),
                       key=os.path.getmtime):
    img = cv2.imread(filename)
    out.write(img)

out.release()